In [1]:
from data import ConnectionString
from IPython.core.interactiveshell import InteractiveShell

import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from urllib.parse import quote_plus
import numpy as np
import seaborn as sns

In [ ]:
# conn_str_builder = ConnectionString()
# conn_str = conn_str_builder.setup_connection_string()
# params = quote_plus(conn_str)
# engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# try:
#     df = pd.read_sql("SELECT * FROM dbo.AttendanceData", engine)

# except Exception as e:
#     print(f"Unable to Connect to SQL Server: {e}")

In [109]:
df = pd.read_csv('CompleteData.csv')

C:\Users\BVM11\AppData\Local\Temp\ipykernel_11132\2546410154.py:1: DtypeWarning: Columns (9,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('CompleteData.csv')


In [3]:
df.info()
InteractiveShell.ast_node_interactivity = 'all' # type: ignore

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2027982 entries, 0 to 2027981
Data columns (total 17 columns):
 #   Column                       Dtype 
---  ------                       ----- 
 0   SCHOOL_YEAR                  object
 1   DISTRICT_CODE                object
 2   DISTRICT_NAME                object
 3   LOCATION_ID                  object
 4   SCHOOL_NAME                  object
 5   STUDENT_ID                   object
 6   Total_Days_Unexcused_Absent  object
 7   Total_Days_Enrolled          object
 8   Total_Days_Present           object
 9   STUDENT_GRADE_LEVEL          object
 10  CURR_GRADE_ORD               object
 11  ETHNIC_CODE                  object
 12  STUDENT_GENDER               object
 13  ECONOMIC_CODE                object
 14  SPECIAL_ED_CODE              object
 15  ENG_PROF_CODE                object
 16  HISPANIC_IND                 object
dtypes: object(17)
memory usage: 263.0+ MB


In [4]:
df = df.drop('CURR_GRADE_ORD', axis=1)
df['SCHOOL_YEAR'] = df['SCHOOL_YEAR'].str.split('-').str[0]
df['STUDENT_GRADE_LEVEL'] = df['STUDENT_GRADE_LEVEL'].replace(['PK', 'KF', 'OS'], [-1, 0, -2])
df['ECONOMIC_CODE'] = df['ECONOMIC_CODE'].replace(['F', 'N', 'R'], [1, 0, 1])
df['SPECIAL_ED_CODE'] = df['SPECIAL_ED_CODE'].replace(['Y', 'N'], [1, 0])
df['ENG_PROF_CODE'] = df['ENG_PROF_CODE'].replace('NT', -1)
df['HISPANIC_IND'] = df['HISPANIC_IND'].replace(['No', 'Yes'], [0, 1])

C:\Users\BVM11\AppData\Local\Temp\ipykernel_11132\2389374588.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['ECONOMIC_CODE'] = df['ECONOMIC_CODE'].replace(['F', 'N', 'R'], [1, 0, 1])
C:\Users\BVM11\AppData\Local\Temp\ipykernel_11132\2389374588.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['SPECIAL_ED_CODE'] = df['SPECIAL_ED_CODE'].replace(['Y', 'N'], [1, 0])
C:\Users\BVM11\AppData\Local\Temp\ipykernel_11132\2389374588.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a fut

In [5]:
non_numeric_columns = ['SCHOOL_NAME', 'DISTRICT_NAME', 'ETHNIC_CODE', 'STUDENT_GENDER']
float_cols = ['Total_Days_Unexcused_Absent', 'Total_Days_Enrolled', 'Total_Days_Present']

numeric_columns = [col for col in df.columns if col not in [*non_numeric_columns, *float_cols]]

for col in numeric_columns:
    df[col] = df[col].astype(int)

for col in float_cols:
    df[col] = df[col].astype(float)

In [6]:
attendance_cols = ['Total_Days_Present', 'Total_Days_Enrolled', 'Total_Days_Unexcused_Absent']
other_cols = [col for col in df.columns if col not in attendance_cols]

agg_dict = {key: 'sum' for key in attendance_cols}
agg_dict.update({key: 'last' for key in other_cols})
grouped_df = df.groupby(by=['STUDENT_ID', 'SCHOOL_YEAR']).agg(agg_dict).reset_index(drop=True)
df = grouped_df[[*df.columns, *grouped_df.columns.difference(df.columns)]]

In [7]:
# df.loc[1688836, 'Total_Days_Present'] = 54.0
# df.loc[731523, 'Total_Days_Present'] = 1.0
# df.loc[1904340, 'Total_Days_Present'] = 2.0
# df.loc[1904340, 'Total_Days_Unexcused_Absent'] = 8.0
# df.loc[1904340, 'Total_Days_Enrolled'] = 10.0
# df.loc[[1016188, 1227776, 1400304, 1401473, 1413193], 'Total_Days_Unexcused_Absent'] = 0.0
# df.loc[686159, 'Total_Days_Unexcused_Absent'] = 3.5
# df.loc[687907, 'Total_Days_Unexcused_Absent'] = 4.0

In [8]:
#12-th grade students
grade_12_students = df[df['STUDENT_GRADE_LEVEL'] == 12]


# Days Enrolled Greater than Unexcused + Present
days_enr_gt_unexcpluspres = df[(df['Total_Days_Unexcused_Absent'] + df['Total_Days_Present']) > df['Total_Days_Enrolled']]

# Grade inconsistencies (demotion of grades)
df_sorted = df.sort_values(by=['STUDENT_ID', 'SCHOOL_YEAR'])
grade_mask = df_sorted.groupby('STUDENT_ID')['STUDENT_GRADE_LEVEL'].diff().lt(0)
grade_inconsitency_df = df[df['STUDENT_ID'].isin(df[grade_mask]['STUDENT_ID'].values)].sort_values(by='STUDENT_ID')

# Drop list
negative_values_indices = [1688836, 731523, 1904340, *[1016188, 1227776, 1400304, 1401473, 1413193], 686159, 687907]

drop_list = [*days_enr_gt_unexcpluspres.index, *grade_inconsitency_df.index, *grade_12_students.index, *negative_values_indices, *df[df['Total_Days_Enrolled'] > 300].index]

df = df.drop(drop_list, axis=0)

In [9]:
mask = df.groupby('STUDENT_ID')['STUDENT_GENDER'].transform('nunique').gt(1)
gender_change_df = df[mask].sort_values(by=['STUDENT_ID', 'SCHOOL_YEAR'])
gender_change_df['STUDENT_GENDER'] = gender_change_df.groupby('STUDENT_ID')['STUDENT_GENDER'].transform('last')

df = df.drop(gender_change_df.index, axis=0)
df = pd.concat([df, gender_change_df], axis=0).reset_index(drop=True)

df['Attendance_Rate'] = df['Total_Days_Present'] / df['Total_Days_Enrolled']

In [10]:
count_series = df.groupby('STUDENT_ID')['SCHOOL_YEAR'].nunique()
training_data = df[df['STUDENT_ID'].isin(count_series[count_series == 6].index)].sort_values(by=['STUDENT_ID', 'SCHOOL_YEAR'])

In [11]:
five_year_data = df[df['STUDENT_ID'].isin(count_series[count_series == 5].index)].sort_values(by=['STUDENT_ID', 'SCHOOL_YEAR'])
non_imputation_data = five_year_data[five_year_data['STUDENT_GRADE_LEVEL'] == -1]
dropping_data = five_year_data[five_year_data['STUDENT_ID'].isin(non_imputation_data['STUDENT_ID'].unique())]

five_year_data = five_year_data.drop(dropping_data.index, axis=0).reset_index(drop=True)

In [12]:
# Checking if linear interpolation is good

def need_non_linear_impute(group, thresh=0.2):
    g = group.set_index('SCHOOL_YEAR').sort_index()
    missing = set([2019, 2020, 2021, 2022, 2023]) - set(g.index)

    for yr in missing:
        if yr - 1 in g.index and yr + 1 in g.index:
            left, right = int(g.loc[[yr - 1, yr + 1], 'Total_Days_Present'].values[0]), int(g.loc[[yr - 1, yr + 1], 'Total_Days_Present'].values[0])
            if abs(right - left) / max(1, max(left, right)) > thresh:
                return True
        
    return False


five_year_data.groupby('STUDENT_ID').apply(need_non_linear_impute).sum()

C:\Users\BVM11\AppData\Local\Temp\ipykernel_11132\1568285024.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  five_year_data.groupby('STUDENT_ID').apply(need_non_linear_impute).sum()


np.int64(0)

In [13]:
# Imputing missing data for five year students using linear interpolation
full_years = range(2019, 2025)
attendance_cols = ['Total_Days_Present', 'Total_Days_Enrolled', 'Total_Days_Unexcused_Absent']
explicit_columns = ['STUDENT_GRADE_LEVEL', 'Attendance_Rate']
static_cols = [col for col in five_year_data.columns if col not in attendance_cols + explicit_columns and col != 'SCHOOL_YEAR']

def impute(group):
    g = group.set_index('SCHOOL_YEAR').sort_index()
    g = g.reindex(full_years)

    g['Is_Imputed'] = g[attendance_cols].isna().any(axis=1)

    g[static_cols] = g[static_cols].ffill().bfill()
    g[attendance_cols] = g[attendance_cols].interpolate(method='linear', limit_direction='both').round(0).astype('float')
    g['Attendance_Rate'] = g['Total_Days_Present'] / group['Total_Days_Enrolled']

    g['STUDENT_GRADE_LEVEL_Check'] = g['STUDENT_GRADE_LEVEL'].ffill().bfill()
    g['STUDENT_GRADE_LEVEL'] = g['STUDENT_GRADE_LEVEL'].fillna(g['STUDENT_GRADE_LEVEL_Check'] + 1)

    return g.reset_index()

five_year_data = five_year_data.groupby('STUDENT_ID').apply(impute)

C:\Users\BVM11\AppData\Local\Temp\ipykernel_11132\3221770233.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  five_year_data = five_year_data.groupby('STUDENT_ID').apply(impute)


In [88]:
# Cleaning Up and Rejoining
five_year_data = five_year_data.reset_index(drop=True)
# five_year_data = five_year_data.drop(['STUDENT_GRADE_LEVEL_Check'], axis=1)
five_year_data['Attendance_Rate'] = five_year_data['Total_Days_Present'] / five_year_data['Total_Days_Enrolled']

integer_cols = [col for col in five_year_data.select_dtypes(include='number') if col not in attendance_cols + ['Attendance_Rate']]
five_year_data[integer_cols] = five_year_data[integer_cols].astype(int)

training_df = pd.concat([training_data, five_year_data], axis=0).sort_values(by=['STUDENT_ID', 'SCHOOL_YEAR']).reset_index(drop=True)
training_df['Is_Imputed'] = training_df['Is_Imputed'].fillna(0).astype(int)

In [ ]:
# Feature Engineering

"""
Columns to Add:

    1) Lag 1, Lag 2 -> Done
    2) Difference in Lags
    3) Expanding and Rolling Averages
    4) Chronic Absence Flags -> Done
    5) School and District Averages
    6) Peer Context
    7) Number of direction changes
    8) Interaction Features
    9) Grade Progression
"""

'\nColumns to Add:\n\n    1) Lag 1, Lag 2\n    2) Difference in Lags\n    3) Expanding and Rolling Averages\n    4) Chronic Absence Flags\n    5) \n'

In [89]:
# Pivoting
attendance_pivot = training_df.pivot(index='STUDENT_ID', columns='SCHOOL_YEAR', values='Attendance_Rate').add_prefix('Attendance_Rate_').reset_index().rename_axis(None, axis=1)

absent_pivot = training_df.pivot(index='STUDENT_ID', columns='SCHOOL_YEAR', values='Total_Days_Unexcused_Absent').add_prefix('Total_Days_Unexcused_Absent_').rename_axis(None, axis=1).reset_index()

present_pivot = training_df.pivot(index='STUDENT_ID', columns='SCHOOL_YEAR', values='Total_Days_Present').add_prefix('Total_Days_Present_').rename_axis(None, axis=1).reset_index()

enrolled_pivot = training_df.pivot(index='STUDENT_ID', columns='SCHOOL_YEAR', values='Total_Days_Enrolled').add_prefix('Total_Days_Enrolled_').rename_axis(None, axis=1).reset_index()

grade_pivot = training_df.pivot(index='STUDENT_ID', columns='SCHOOL_YEAR', values='STUDENT_GRADE_LEVEL').add_prefix('STUDENT_GRADE_LEVEL_').rename_axis(None, axis=1).reset_index()

imputed_pivot = training_df.pivot(index='STUDENT_ID', columns='SCHOOL_YEAR', values='Is_Imputed').add_prefix('Is_Imputed_').rename_axis(None, axis=1).reset_index()

In [90]:
non_aggregate_cols = ['Total_Days_Unexcused_Absent', 'Total_Days_Enrolled', 'Total_Days_Present', 'STUDENT_GRADE_LEVEL', 'Attendance_Rate', 'Is_Imputed']

aggreagte_cols = [col for col in training_df.columns if col not in non_aggregate_cols]
last_agg_dict = {key: 'last' for key in aggreagte_cols}

aggregated_df = training_df.groupby('STUDENT_ID').agg(last_agg_dict).reset_index(drop=True)

In [91]:
# Merging Pivot Columns into a dataframe
training_df = pd.concat([attendance_pivot.iloc[:, 1:], absent_pivot.iloc[:, 1:], present_pivot.iloc[:, 1:], enrolled_pivot.iloc[:, 1:], grade_pivot.iloc[:, 1:], imputed_pivot.iloc[:, 1:], aggregated_df], axis=1)

In [103]:
def build_features(year):
    training_df[f'Lag_1_{year}'] = training_df[f'Attendance_Rate_{year - 1}'] if f'Attendance_Rate_{year - 1}' in training_df.columns else np.nan

    training_df[f'Lag_2_{year}'] = training_df[f'Attendance_Rate_{year - 2}'] if f'Attendance_Rate_{year - 2}' in training_df.columns else np.nan

    training_df[f'Is_Chronically_Absent_{year}'] = (training_df[f'Attendance_Rate_{year}'] < 0.85).astype(int)

    training_df[f'School_Average_{year}'] = training_df.groupby('LOCATION_ID')[f'Attendance_Rate_{year}'].median()

    training_df[f'District Average_{year}'] = training_df.groupby('DISTRICT_CODE')[f'Attendance_Rate_{year}'].median()

    training_df[f'Grade_Average_{year}'] = training_df.groupby(f'STUDENT_GRADE_LEVEL_{year}')[f'Attendance_Rate_{year}'].median()

for year in range(2019, 2025):
    build_features(year)

In [108]:
training_df.groupby(by=['LOCATION_ID', f'STUDENT_GRADE_LEVEL_2020'])[f'Attendance_Rate_2020'].describe()

count      mean       std       min  \
LOCATION_ID STUDENT_GRADE_LEVEL_2020                                         
1           -1                          91.0  0.896823  0.195111  0.000000   
             0                        1092.0  0.942599  0.058460  0.457831   
             1                        1384.0  0.949941  0.048971  0.600000   
             2                        2077.0  0.960381  0.040829  0.568421   
             3                        2248.0  0.959857  0.047515  0.000000   
...                                      ...       ...       ...       ...   
952          1                          22.0  0.988451  0.025953  0.895833   
             2                          20.0  0.996443  0.011059  0.959627   
             3                          17.0  0.998190  0.007463  0.969231   
             4                          16.0  0.991382  0.019640  0.924812   
             5                           2.0  1.000000  0.000000  1.000000   

                                           25%       50%       75%  max  
LOCATION_ID STUDENT_GRADE_LEVEL_2020                                     
1           -1                        0.891447  0.955882  1.000000  1.0  
             0                        0.924242  0.955390  0.984158  1.0  
             1                        0.932331  0.961538  0.984526  1.0  
             2                        0.943503  0.970833  0.991870  1.0  
             3                        0.942434  0.970803  0.992308  1.0  
...                                        ...       ...       ...  ...  
952          1                        0.994231  1.000000  1.000000  1.0  
             2                        1.000000  1.000000  1.000000  1.0  
             3                        1.000000  1.000000  1.000000  1.0  
             4                        0.992018  1.000000  1.000000  1.0  
             5                        1.000000  1.000000  1.000000  1.0  

[2039 rows x 8 columns]